In [3]:
from flux_utils import flux_conversion
from astropy import units as u
import numpy as np

In [4]:
x = [200 * 10**12, 10*10.**15]
ul_txs_6_month_flux = 5. * 10 **-11. * u.erg / (u.cm**2 * u.s)
ul_txs_6_month = ul_txs_6_month_flux /(flux_conversion/u.cm**2)
ratio_ehe_alert = (6.6 + 2.8) / 2.1 # ICRC REALTIME PAPER RATIO GOLD TO EHE
ul_6_month_alert = ul_txs_6_month / ratio_ehe_alert

In [7]:
print(ul_txs_6_month * flux_conversion * (0.5 * u.year).to("s"))

0.0007889399999999999 erg


In [8]:
print(ratio_ehe_alert)

4.476190476190475


In [12]:
print(ul_6_month_alert*(0.5 * u.year).to("s"))

1.1679870814766889e+51 erg


In [6]:
ul_e = ul_6_month_alert * (0.5 * u.year).to("s")

In [7]:
print(ul_e)

1.1679870814766889e+51 erg
